In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import LeaveOneGroupOut
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"

physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.set_visible_devices(physical_devices[0], 'GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [11]:
df = pd.read_pickle(os.getcwd() + '/merged_data.pkl')
df.head(20)

,UnixTimeStamp,X,Y,PupilSize,Item,CurrentClickX,CurrentClickY,CurrentURL,pid
0,1.714461e+12,692.5,662.5,1428.0,Sunscreen,NaN,NaN,NaN,21
1,1.714461e+12,690.0,662.5,1428.0,Sunscreen,NaN,NaN,NaN,21
2,1.714461e+12,689.0,667.0,1426.0,Sunscreen,NaN,NaN,NaN,21
3,1.714461e+12,690.5,666.0,1425.0,Sunscreen,NaN,NaN,NaN,21
4,1.714461e+12,690.5,666.0,1425.0,Sunscreen,NaN,NaN,NaN,21
5,1.714461e+12,690.0,666.0,1426.0,Sunscreen,NaN,NaN,NaN,21
6,1.714461e+12,688.5,666.5,1422.0,Sunscreen,NaN,NaN,NaN,21
7,1.714461e+12,688.0,666.5,1421.0,Sunscreen,NaN,NaN,NaN,21
8,1.714461e+12,688.0,666.5,1421.0,Sunscreen,NaN,NaN,NaN,21
9,1.714461e+12,688.0,667.0,1421.0,Sunscreen,NaN,NaN,NaN,21


In [12]:
print(len(df))

37910626


In [13]:
'''
TODO: 
- Create windows for your LSTM model
    - Use X, Y and PupilSize as input
    - Use CurrentClickX and CurrentClickY as labels (regression)
    - Select your own window size and stride
- Split the data participant wise (on pid) 
    - https://sven-mayer.com/pml/files/12_Split.pdf
    - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html
    - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneGroupOut.html
    - Several options to split the data, Leave One Out, Leave One Group Out, all are valid options. You can pick :)
- Build a LSTM model
    - Use the Sequential API
    - Use LSTM layers
    - Make sure you have a 2-neuron regression as output!
        - https://pieriantraining.com/tensorflow-lstm-example-a-beginners-guide/
        - https://towardsdatascience.com/lstm-by-example-using-tensorflow-feb0c1968537
        
'''

'\nTODO: \n- Create windows for your LSTM model\n    - Use X, Y and PupilSize as input\n    - Use CurrentClickX and CurrentClickY as labels (regression)\n    - Select your own window size and stride\n- Split the data participant wise (on pid) \n    - https://sven-mayer.com/pml/files/12_Split.pdf\n    - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html\n    - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneGroupOut.html\n    - Several options to split the data, Leave One Out, Leave One Group Out, all are valid options. You can pick :)\n- Build a LSTM model\n    - Use the Sequential API\n    - Use LSTM layers\n    - Make sure you have a 2-neuron regression as output!\n        - https://pieriantraining.com/tensorflow-lstm-example-a-beginners-guide/\n        - https://towardsdatascience.com/lstm-by-example-using-tensorflow-feb0c1968537\n        \n'

In [14]:
unique_users = df['pid'].unique()
test_users = np.random.choice(unique_users, size=10, replace=False)

train_df = df[~df['pid'].isin(test_users)]
test_df = df[df['pid'].isin(test_users)]

print(len(train_df))
print(len(test_df))

27627594
10283032


In [17]:
def create_windows(df, columns_X, columns_y, window_size, stride):
    windows = []
    labels = []
    
    for i in range(0, len(df) - window_size + 1, stride):
        window = df.iloc[i:i + window_size][columns_X].copy()
        windows.append(window)
        
        label = df.iloc[i:i + window_size][columns_y].copy()
        labels.append(label)
        
    return windows, labels

In [18]:
columns_X = ['X', 'Y', 'PupilSize']
columns_y = ['CurrentClickX', 'CurrentClickY']
window_size = 500000
stride = 100000

In [19]:
X_train, y_train = create_windows(train_df, columns_X, columns_y, window_size, stride)
X_test, y_test = create_windows(test_df, columns_X, columns_y, window_size, stride)

In [20]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(50, return_sequences=True, input_shape=(window_size, 1)),
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dense(2)
])

/Users/khang.van/PycharmProjects/practical-machine-learning/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [21]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

In [22]:
history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test), verbose=2)

Epoch 1/20


KeyboardInterrupt: 

In [ ]:
# Plot the loss and accuracy
plt.figure(figsize=(12, 6))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Accuracy plot
if 'accuracy' in history.history:
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

plt.show()